<a href="https://colab.research.google.com/github/luizaribeiro06/data-science/blob/main/cp2_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as Path
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/scouts_2015.csv')

print("Dimensão da base: ", df.shape)
df.head()

Dimensão da base:  (29720, 34)


,atleta_id,apelido,rodada_id,ano,clube_id,clube,posicao_id,posicao,jogos_num,pontos_num,...,FC,GC,CA,CV,SG,DD,DP,GS,status_id,status
0,87552,Vitor Bueno,27,2015,277,Santos,4,mei,1,0.0,...,0,0,0,0,0,0,0,0,0,NaN
1,87552,Vitor Bueno,28,2015,277,Santos,4,mei,1,0.0,...,0,0,0,0,0,0,0,0,0,NaN
2,87552,Vitor Bueno,29,2015,277,Santos,4,mei,1,0.0,...,0,0,0,0,0,0,0,0,0,NaN
3,87552,Vitor Bueno,30,2015,277,Santos,4,mei,1,0.0,...,0,0,0,0,0,0,0,0,0,NaN
4,87552,Vitor Bueno,31,2015,277,Santos,4,mei,2,-0.3,...,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
#pontuação média dos jogadores nas rodadas (pegar o dado de uma)
df.groupby("rodada_id")["pontos_num"].mean()

,pontos_num
rodada_id,
1,0.883538
2,1.055750
3,0.886453
4,0.984207
5,1.098376
6,0.891890
7,0.728041
8,0.680588
9,0.835067


In [ ]:
# Qual posição tem a maior mediana de pontos?
mediana_por_posicao = df.groupby("posicao")["pontos_num"].median().sort_values(ascending=False)

print("Mediana de pontos por posição:")
display(mediana_por_posicao)

Mediana de pontos por posição:


,pontos_num
posicao,
tec,2.88
ata,0.00
gol,0.00
lat,0.00
mei,0.00
zag,0.00


In [ ]:
# Jogador que mais aparece (moda)
moda = df["apelido"].mode()[0]

# Quantas vezes esse jogador aparece
contagem = df["apelido"].value_counts()[moda]

print(f"O jogador que mais aparece é {moda}, com {contagem} registros.")


O jogador que mais aparece é Juninho, com 203 registros.


In [ ]:
# pontuação dos atacantes é mais dispersa que a dos zagueiros?
atacantes = df[df["posicao"] == "ata"]["pontos_num"]
zagueiros = df[df["posicao"] == "zag"]["pontos_num"]

# calculando a dispersão (desvio padrão)
disp_atacantes = atacantes.std()
disp_zagueiros = zagueiros.std()

print(f"Desvio padrão Atacantes: {disp_atacantes:.2f}")
print(f"Desvio padrão Zagueiros: {disp_zagueiros:.2f}")


Desvio padrão Atacantes: 3.12
Desvio padrão Zagueiros: 2.64


In [ ]:
# qual posição tem maior consistência de desempenho (menor desvio-padrão de pontos)?

disp_por_posicao = df.groupby("posicao")["pontos_num"].std().sort_values()

display(disp_por_posicao)

posicao_consistente = disp_por_posicao.idxmin()
valor_consistente = disp_por_posicao.min()

,pontos_num
posicao,
tec,2.197947
mei,2.387970
zag,2.640652
lat,2.771329
gol,3.077806
ata,3.120259


In [ ]:
# Verificar se existe a coluna de preços
print(df.columns)

# Calcular desvio padrão do preço por posição
variabilidade_preco = df.groupby("posicao")["preco_num"].std().sort_values()

print("Variabilidade (desvio-padrão) do preço por posição:")
display(variabilidade_preco)

Index(['atleta_id', 'apelido', 'rodada_id', 'ano', 'clube_id', 'clube',
       'posicao_id', 'posicao', 'jogos_num', 'pontos_num', 'media_num',
       'preco_num_anterior', 'preco_num', 'variacao_num', 'FS', 'PE', 'A',
       'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG',
       'DD', 'DP', 'GS', 'status_id', 'status'],
      dtype='object')
Variabilidade (desvio-padrão) do preço por posição:


,preco_num
posicao,
tec,2.862776
zag,4.384655
lat,4.422924
mei,4.591504
ata,5.491849
gol,6.076104


In [ ]:
# converter a coluna de pontos para numérico
df['pontos_num'] = pd.to_numeric(df['pontos_num'], errors='coerce')

# calcular limites do IQR
Q1, Q3 = df['pontos_num'].quantile([0.25, 0.75])
IQR = Q3 - Q1
limite_sup = Q3 + 1.5 * IQR

# filtrar outliers (pontuações muito altas)
outliers = df[df['pontos_num'] > limite_sup]

# mostrar top 10 maiores pontuações
print("Jogadores com pontuação extremamente alta:")
display(outliers[['apelido', 'rodada_id', 'pontos_num']].sort_values(by='pontos_num', ascending=False).head(10))



Jogadores com pontuação extremamente alta:


,apelido,rodada_id,pontos_num
10519,Willian,23,32.7
4868,Gabriel,37,25.6
951,Alexandre Pato,13,25.1
9147,Erik,19,24.1
8259,Vitinho,13,23.4
8796,Léo Gamalho,21,23.1
7823,Giuliano,5,22.7
16489,Weverton,32,22.7
6468,Alan Patrick,34,22.5
426,Zé Roberto,4,22.3


In [ ]:
# converter colunas para numérico
df['pontos_num'] = pd.to_numeric(df['pontos_num'], errors='coerce')
df['preco_num'] = pd.to_numeric(df['preco_num'], errors='coerce')

# calcular limites normais com IQR
Q1_pontos, Q3_pontos = df['pontos_num'].quantile([0.25, 0.75])
IQR_pontos = Q3_pontos - Q1_pontos
Q1_preco, Q3_preco = df['preco_num'].quantile([0.25, 0.75])
IQR_preco = Q3_preco - Q1_preco

limite_sup_preco = Q3_preco + 1.5 * IQR_preco
limite_inf_preco = Q1_preco - 1.5 * IQR_preco

# jogadores destoantes
caros_baixos = df[(df['preco_num'] > limite_sup_preco) & (df['pontos_num'] < Q1_pontos)]
baratos_altos = df[(df['preco_num'] < limite_inf_preco) & (df['pontos_num'] > Q3_pontos)]

if not caros_baixos.empty or not baratos_altos.empty:
    print("Sim, houve jogadores cujo preço destoou da pontuação real.")
else:
    print("Não houve jogadores com preço destoante da pontuação.")


Sim, houve jogadores cujo preço destoou da pontuação real.


In [ ]:

# filtra apenas goleiros
goleiros = df[df['posicao'] == 'gol']

# detecta outliers em DD e DP (método IQR)
Q1_DD = goleiros['DD'].quantile(0.25)
Q3_DD = goleiros['DD'].quantile(0.75)
IQR_DD = Q3_DD - Q1_DD

Q1_DP = goleiros['DP'].quantile(0.25)
Q3_DP = goleiros['DP'].quantile(0.75)
IQR_DP = Q3_DP - Q1_DP

# limites
limite_inf_DD = Q1_DD - 1.5 * IQR_DD
limite_sup_DD = Q3_DD + 1.5 * IQR_DD
limite_inf_DP = Q1_DP - 1.5 * IQR_DP
limite_sup_DP = Q3_DP + 1.5 * IQR_DP

# goleiros fora do esperado em DD ou DP
fora_esperado = goleiros[
    (goleiros['DD'] < limite_inf_DD) | (goleiros['DD'] > limite_sup_DD) |
    (goleiros['DP'] < limite_inf_DP) | (goleiros['DP'] > limite_sup_DP)
]

# exibir resultado
if fora_esperado.empty:
    print("Nenhum goleiro com desempenho fora do esperado em defesas (DD, DP).")
else:
    print("Goleiros com desempenho fora do esperado em defesas (DD, DP):")
    print(fora_esperado[['apelido', 'DD', 'DP']])




Goleiros com desempenho fora do esperado em defesas (DD, DP):
              apelido  DD  DP
267            Magrão   1   0
533             César   2   0
535             César   1   0
536             César   1   0
537             César   2   0
...               ...  ..  ..
19929  Fernando Prass   1   0
29600         Charles   1   0
29626    Bruno Grassi   1   0
29681           Jordi   1   0
29686           Jordi   1   0

[555 rows x 3 columns]
